In [1]:
import geopandas as gpd
import pandas as pd
from scipy.spatial import distance_matrix
import numpy as np
import geopandas as gpd
import re


# If you need a JSON string
import json


In [2]:
demand = pd.read_csv('../data/curated/train_demand/daily_trains_demand_post_covid.csv')
stations = gpd.read_file('../data/station_and_polygon_centre_data/station and polygon centre data/all_points_postcovid_(SA2 without stations).shp')
stations = stations.to_crs("EPSG:7855")

In [3]:
def get_weight(df, var=None):
    df = df.sort_values('Point Name')
    df['Point Name'] = df['Point Name'].apply(lambda x: re.sub('~', '', x))
    coords = df['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist()
    dist_matrix = distance_matrix(coords, coords)
    dist_df = pd.DataFrame(dist_matrix, index=list(df['Point Name']), columns=list(df['Point Name']))
    if var:
        variance = var
    else:
        variance = np.var(dist_matrix.flatten())
    W = np.exp(-1 * np.power(dist_df,2) / (2 * variance)) - np.eye(len(df))
    dict_data = W.to_dict(orient='index')
    return W, dict_data, variance

In [4]:
W, dict_stations, variance = get_weight(stations[stations['point_type'] == 'station'])
with open('../output/station_weights.json', 'w', encoding='utf-8') as f:
    json.dump(dict_stations, f, ensure_ascii=False, indent=4)
W.to_csv('../output/station_weights.csv')

In [5]:
stations_self = stations[stations['point_type'] == 'station']
stations_missing = stations[stations['point_type'] == 'suburb']
stations_missing['Point Name'] = stations_missing['Point Name'].apply(lambda x: '~(SA2)' + x)
stations_and_SA2 = pd.concat([stations_self, stations_missing])

c:\Users\Lenovo\anaconda3\envs\spanalytics\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
W, dict_stations_and_SA2, variance = get_weight(stations_and_SA2, variance)
with open('../output/station_weights_withSA2.json', 'w', encoding='utf-8') as f:
    json.dump(dict_stations, f, ensure_ascii=False, indent=4)
W.to_csv('../output/station_weights_withSA2.csv')

In [7]:
W

,Aircraft,Alamein,Albion,Alphington,Altona,Anstey,Armadale,Ascot Vale,Ashburton,Aspendale,...,(SA2)Warrandyte - Wonga Park,(SA2)Werribee - East,(SA2)Werribee - West,(SA2)West Melbourne - Residential,(SA2)Wheelers Hill,(SA2)Whittlesea,(SA2)Wollert,(SA2)Wyndham Vale - North,(SA2)Wyndham Vale - South,(SA2)Yarra Valley
Aircraft,0.000000e+00,0.194242,0.764265,0.251606,0.926457,0.394715,0.339595,0.531426,0.190774,0.079223,...,0.015872,8.467697e-01,4.500334e-01,0.515655,0.051447,0.004648,0.054235,5.176241e-01,7.065439e-01,9.261661e-07
Alamein,1.942415e-01,0.000000,0.283733,0.782567,0.365150,0.593051,0.939621,0.535870,0.998920,0.519116,...,0.435054,5.967828e-02,9.930122e-03,0.695134,0.798792,0.029417,0.127041,1.336463e-02,3.132166e-02,2.289980e-03
Albion,7.642653e-01,0.283733,0.000000,0.501922,0.813980,0.736585,0.462744,0.858525,0.287873,0.058612,...,0.057429,4.848741e-01,1.741603e-01,0.757956,0.078219,0.038638,0.241437,3.161386e-01,3.617938e-01,7.292671e-06
Alphington,2.516064e-01,0.782567,0.501922,0.000000,0.424034,0.915741,0.853379,0.823878,0.802843,0.188497,...,0.469620,8.267851e-02,1.397434e-02,0.877560,0.441125,0.122660,0.401574,2.813440e-02,4.566695e-02,1.105206e-03
Altona,9.264574e-01,0.365150,0.813980,0.424034,0.000000,0.568462,0.558267,0.702261,0.359933,0.157016,...,0.042786,6.303347e-01,2.607580e-01,0.725624,0.122767,0.009118,0.085894,3.101245e-01,4.758640e-01,6.600033e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
(SA2)Whittlesea,4.647562e-03,0.029417,0.038638,0.122660,0.009118,0.122222,0.033140,0.082127,0.033195,0.000759,...,0.155168,9.288617e-04,7.330916e-05,0.057818,0.012037,0.000000,0.687648,5.807557e-04,4.355491e-04,6.811636e-04
(SA2)Wollert,5.423491e-02,0.127041,0.241437,0.401574,0.085894,0.460325,0.163434,0.373775,0.138342,0.006515,...,0.220901,1.616726e-02,2.150162e-03,0.282402,0.044803,0.687648,0.000000,1.041370e-02,8.911129e-03,2.783210e-04
(SA2)Wyndham Vale - North,5.176241e-01,0.013365,0.316139,0.028134,0.310124,0.071391,0.035033,0.119492,0.013152,0.003445,...,0.000499,7.880174e-01,7.989453e-01,0.090887,0.001663,0.000581,0.010414,0.000000e+00,8.672793e-01,1.734388e-09
(SA2)Wyndham Vale - South,7.065439e-01,0.031322,0.361794,0.045667,0.475864,0.097443,0.071435,0.159772,0.030294,0.013894,...,0.001034,9.672610e-01,9.058157e-01,0.141591,0.005287,0.000436,0.008911,8.672793e-01,0.000000e+00,8.165618e-09
